# 📘 ADM1 Model Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/benmola/OpenAD-lib/blob/main/notebooks/01_ADM1_Tutorial.ipynb)

This notebook demonstrates the **Anaerobic Digestion Model No. 1 (ADM1)** implementation in OpenAD-lib.

---

## 📚 References

- **ADM1 Implementation**: [PyADM1 GitHub](https://github.com/CaptainFerMag/PyADM1)
- **ADM1 Paper**: [Rosén & Jeppsson (2021) - BioRxiv](https://www.biorxiv.org/content/biorxiv/early/2021/03/04/2021.03.03.433746.full.pdf)
- **ACoD Feedstock Characterization**: [Astals et al. (2015) - PubMed](https://pubmed.ncbi.nlm.nih.gov/27088248/)

## 🔬 ADM1 Background

The Anaerobic Digestion Model No. 1 (ADM1) is an internationally recognized standard for modelling anaerobic digestion processes.

### Model Structure

ADM1 describes the conversion of organic matter into biogas through a series of biochemical and physicochemical processes:

**Biochemical Processes:**
1. **Disintegration**: Complex particulates → Carbohydrates, Proteins, Lipids
2. **Hydrolysis**: Polymers → Monomers (sugars, amino acids, LCFA)
3. **Acidogenesis**: Monomers → VFAs (acetate, propionate, butyrate, valerate)
4. **Acetogenesis**: VFAs → Acetate + H₂
5. **Methanogenesis**: Acetate/H₂ → CH₄ + CO₂

### Key Equations

**Monod Kinetics for Substrate Uptake:**

$$\rho_j = k_{m,j} \cdot \frac{S_j}{K_{S,j} + S_j} \cdot X_j \cdot I_{pH,j} \cdot I_{IN,j}$$

Where:
- $\rho_j$ = uptake rate for process $j$
- $k_{m,j}$ = maximum specific uptake rate
- $K_{S,j}$ = half-saturation constant
- $I_{pH}$, $I_{IN}$ = inhibition functions

**Gas-Liquid Transfer:**

$$\rho_{T,i} = k_{L}a \cdot (S_i - K_{H,i} \cdot p_{gas,i})$$

**State Variables (35+ states):**
- 12 Soluble components (sugars, amino acids, LCFA, VFAs, gases)
- 13 Particulate components (biomass, inerts)
- 6 Ion states (for pH calculation)
- 3 Gas phase components (CH₄, CO₂, H₂)

## 1️⃣ Setup (Google Colab)

Run this cell to install OpenAD-lib in Google Colab:

In [ ]:
# Install OpenAD-lib from GitHub (uncomment for Colab)
# !pip install "git+https://github.com/benmola/OpenAD-lib.git#egg=openad_lib"


# For local development, add src to path
import sys
import os

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:
    # Local: add src to path
    sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

print(f"Running in Colab: {IN_COLAB}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openad_lib.models.mechanistic import ADM1Model
from openad_lib.preprocessing import acod

print("✅ All imports successful!")

## 2️⃣ Feedstock Characterization (ACoD)

The **Anaerobic Co-Digestion (ACoD)** method converts feedstock mixture ratios into detailed ADM1 state variables.

### Method
Based on [Astals et al. (2015)](https://pubmed.ncbi.nlm.nih.gov/27088248/), feedstock properties are mapped to:
- Carbohydrate, protein, lipid fractions
- Particulate vs. soluble fractions
- BMP (Biochemical Methane Potential)

In [ ]:
# Define data paths
if IN_COLAB:
    # Download sample data for Colab
    !wget -q https://raw.githubusercontent.com/benmola/OpenAD-lib/main/src/openad_lib/data/feedstock/Feed_Data.csv
    !wget -q https://raw.githubusercontent.com/benmola/OpenAD-lib/main/src/openad_lib/data/Biogas_Plant_Outputs.csv
    ratio_file = 'Feed_Data.csv'
    measured_file = 'Biogas_Plant_Outputs.csv'
else:
    base_path = os.path.dirname(os.getcwd())
    ratio_file = os.path.join(base_path, 'src', 'openad_lib', 'data', 'feedstock', 'Feed_Data.csv')
    measured_file = os.path.join(base_path, 'src', 'openad_lib', 'data', 'Biogas_Plant_Outputs.csv')

# Generate influent data
print("Generating ADM1 influent from feedstock ratios...")
influent_df = acod.generate_influent_data(ratio_file)

# Load measured biogas data
measured_df = pd.read_csv(measured_file)
print(f"\n📊 Loaded measured data: {len(measured_df)} days")

print(f"\nInfluent shape: {influent_df.shape}")
influent_df.head()

## 3️⃣ ADM1 Model Initialization

The `ADM1Model` class implements the complete ADM1 differential equation system with:
- 35+ state variables
- BSM2 default parameters
- ODE solver with adaptive stepping

In [ ]:
# Initialize ADM1 model
model = ADM1Model()

print("📊 ADM1 Model Initialized")
print(f"   State variables: 35+")
print(f"   Processes: 19 biochemical + physicochemical")

## 4️⃣ Run Simulation

The simulation integrates the ODE system:

$$\frac{dS_i}{dt} = \frac{Q_{in}}{V_{liq}}(S_{i,in} - S_i) + \sum_j \nu_{i,j} \cdot \rho_j$$

Where:
- $Q_{in}$ = influent flow rate
- $V_{liq}$ = liquid volume
- $\nu_{i,j}$ = stoichiometric coefficients
- $\rho_j$ = process rates

In [ ]:
print("🚀 Starting simulation...")
simulation_output = model.simulate(influent_df)

# Extract results
results = simulation_output['results']
q_gas_df = simulation_output['q_gas']

print(f"✅ Simulation complete!")
print(f"   Duration: {results['time'].max():.0f} days")
print(f"   Mean simulated biogas: {q_gas_df['q_gas'].mean():.2f} m³/day")
print(f"   Mean measured biogas: {measured_df['Biogas (m3/day)'].mean():.2f} m³/day")

## 5️⃣ Compare Simulated vs Measured Biogas

In [ ]:
plt.style.use('bmh')
fig, ax = plt.subplots(figsize=(14, 6))

# Simulated biogas
ax.plot(q_gas_df['time'], q_gas_df['q_gas'], 
        label='Simulated (ADM1)', linewidth=2, color='#E67E22')

# Measured biogas
ax.plot(measured_df['time'], measured_df['Biogas (m3/day)'], 
        'o', markersize=4, alpha=0.6, color='#2E86C1', label='Measured')

ax.set_xlabel('Time (days)', fontsize=14, fontweight='bold')
ax.set_ylabel('Flow Rate (m³/day)', fontsize=14, fontweight='bold')
ax.set_title('ADM1 Biogas Production: Simulated vs Measured', fontsize=16, pad=20)
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Calculate metrics
from sklearn.metrics import mean_squared_error, r2_score
common_len = min(len(q_gas_df), len(measured_df))
rmse = np.sqrt(mean_squared_error(
    measured_df['Biogas (m3/day)'].values[:common_len],
    q_gas_df['q_gas'].values[:common_len]
))
print(f"📈 RMSE: {rmse:.2f} m³/day")

## 6️⃣ VFA Dynamics

VFAs (Volatile Fatty Acids) are intermediate products in AD. Key VFAs include:
- **Acetate** (S_ac) - main precursor for methanogenesis
- **Propionate** (S_pro) - slower degradation
- **Butyrate** (S_bu) - intermediate degradation

In [ ]:
# VFA Dynamics
fig, ax = plt.subplots(figsize=(14, 6))

# Convert to g/L if values are in kg/m³ (standard ADM1 units are kg COD/m³)
# Typical VFA concentrations: 0.01 - 5 kg COD/m³
ax.plot(results['time'], results['S_ac'], label='Acetate', linewidth=2)
ax.plot(results['time'], results['S_pro'], label='Propionate', linewidth=2)
ax.plot(results['time'], results['S_bu'], label='Butyrate', linewidth=2)

ax.set_xlabel('Time (days)', fontsize=14, fontweight='bold')
ax.set_ylabel('Concentration (kg COD/m³)', fontsize=14, fontweight='bold')
ax.set_title('VFA Dynamics in Digester', fontsize=16, pad=20)
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

# Display current VFA levels
print(f"📊 Final VFA Concentrations:")
print(f"   Acetate:    {results['S_ac'].iloc[-1]:.4f} kg COD/m³")
print(f"   Propionate: {results['S_pro'].iloc[-1]:.4f} kg COD/m³")
print(f"   Butyrate:   {results['S_bu'].iloc[-1]:.4f} kg COD/m³")

plt.tight_layout()
plt.show()

## 📝 Summary

In this notebook, we demonstrated:

1. **ACoD Preprocessing** - Converting feedstock ratios to ADM1 influent states
2. **ADM1 Simulation** - Running the full 35+ state ODE system
3. **Model Validation** - Comparing simulated vs measured biogas production
4. **VFA Analysis** - Monitoring process stability indicators

### Next Steps

- Try the [AM2 Simplified Model](02_AM2_Modelling.ipynb) for faster simulations
- Explore [LSTM Surrogate Models](03_LSTM_Prediction.ipynb) for real-time prediction
- Use [Multi-Task GP](04_MTGP_Prediction.ipynb) for uncertainty quantification